# Cross sections
Plot our elastic and inelastic scattering cross-sections for hydrogen/tritium. Look angular distributions of outgoing particles.

In [ ]:
# Import common libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as const

In [ ]:
# Make font size bigger and allow LaTeX rendering
plt.rcParams.update({'font.size': 15})
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

The ionisation cross-sections are from M. E. Rudd, *Phys. Rev. A* **44** (1991).

In [ ]:
# Import cross-section models
from src.Rudd1991 import RuddXSec
from src.Mott import MottXSec
import src.Constants as const

## Total cross-sections
See what our total cross-sections look like across the tritium $\beta$-decay spectrum.

In [ ]:
incidentEArray = np.linspace(1e3, 19e3, 300)  # Incident electron energies in eV
mottXSecArray = np.zeros_like(incidentEArray)
ionisationXSecArray = np.zeros_like(incidentEArray)
for iE, E in enumerate(incidentEArray):
    mottXSecArray[iE] = MottXSec(E).TotalXSec()
    ionisationXSecArray[iE] = RuddXSec(E).TotalXSec()

# Plot the cross-sections
plt.figure()
plt.plot(incidentEArray / 1e3, mottXSecArray * 1e4, label='Mott')
plt.plot(incidentEArray / 1e3, ionisationXSecArray * 1e4, label='Ionisation')
plt.yscale('log')
plt.xlabel('Incident electron energy [keV]')
plt.ylabel('Cross-section [cm$^2$]')
plt.legend()
plt.show()

## Angular distributions for outgoing electrons

In [ ]:
endpointEnergy = 18.575e3 # eV
endpointElasticXSec = MottXSec(endpointEnergy)
endpointIonisationXSec = RuddXSec(endpointEnergy)

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
angles = np.linspace(1e-5, np.pi - 1e-4, 600)
plt.plot(angles, endpointElasticXSec.SinglyDifferentialXSec_theta(angles) * 1e4)
plt.xlabel('Scattering angle [radians]')
plt.yscale('log')
plt.ylabel('$d \sigma / d \\rm{cos}\\theta$ [cm$^2$]')
plt.title(f'Elastic scattering, T = {endpointEnergy / 1e3:.3f} keV')
plt.xlim([-0.01, np.pi])

plt.subplot(1, 2, 2)
plt.plot(angles, endpointIonisationXSec.DoublyDifferentialXSec(endpointEnergy - const.IONIZATIONENERGYH - 1, angles) * 1e4, label='1 eV')
plt.plot(angles, endpointIonisationXSec.DoublyDifferentialXSec(endpointEnergy - const.IONIZATIONENERGYH - 5, angles) * 1e4, label='5 eV')
plt.plot(angles, endpointIonisationXSec.DoublyDifferentialXSec(endpointEnergy - const.IONIZATIONENERGYH - 20, angles) * 1e4, label='20 eV')
plt.plot(angles, endpointIonisationXSec.DoublyDifferentialXSec(endpointEnergy - const.IONIZATIONENERGYH - 50, angles) * 1e4, label='50 eV')
plt.yscale('log')
plt.xlabel('Scattering angle [radians]')
plt.ylabel('$d^2 \sigma / (d \\rm{cos}\\theta dW)$ [cm$^2$ eV$^{-1}$]')
plt.xlim([0, np.pi / 4])
plt.legend(title = 'Secondary energy')
plt.title(f'Ionisation, T = {endpointEnergy / 1e3:.3f} keV')
plt.show()